# TODOs

- ✓ def solve => def get_instance, returner problem i stedet for løsning. Bl.a. mulighed for at få feasibility.
- ✓ def get_task_counts, laver stats matrix
- ✓ arg sick => arg hu + arg sick
- ✓ arg df => arg task_counts
- ✓ arg assigned
- ✓ løs for hel dag
- ✓ tilføj constraints i prob
- ✓ tilføj allerede assigned constraint
- tilføj vagthavende constraint


## Create synthetic data

In [151]:
MAX_SICK = 2
N_GASTS = 60
N_TEAMS = 3
N_SHIFTS = 6
N_TASKS = len(Task)
TEAM_SIZE = 20
TEAM_ORDER = [0,1,2]
N_GASTS = 60
N_TEAMS = 3
N_SHIFTS = 6
N_TASKS = len(Task)
TEAM_SIZE = 20
TEAM_ORDER = [0,1,2]
KABYS_TIDER = [0,1,2]
PEJLE_SHIFT = 3


dates = pd.date_range(start='3/1/2021', end='4/14/2021').date

history = []
np.random.seed(42)

for dt in dates:
    for shift in range(N_SHIFTS):
        n_sick = np.random.randint(0,MAX_SICK)
        sick = np.random.choice(N_GASTS, n_sick)
        team = TEAM_ORDER[shift % N_TEAMS]
        min_gast, max_gast = team_minmax(team)
        active_gasts = [gast for gast in range(min_gast, max_gast) if gast not in sick]
        for task_no, gast_no in enumerate(np.random.permutation(active_gasts)[:N_TASKS]):
            task_name = Task(task_no).name
            entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': task_no, 'task_name': task_name}
            history.append(entry)
        for gast_no in sick:
            entry = {'date': dt, 'shift': shift, 'gast_no': gast_no, 'task_no': -1, 'task_name': 'SICK'}
            history.append(entry)
df = pd.DataFrame(history)
df.to_csv('../data/synthetic.csv', index=False)
df

,date,shift,gast_no,task_no,task_name
0,2021-03-01,0,0,0,VAGTHAVENDE_ELEV
1,2021-03-01,0,17,1,ORDONNANS
2,2021-03-01,0,15,2,UDKIG
3,2021-03-01,0,1,3,BJÆRGEMÆRS
4,2021-03-01,0,8,4,RORGÆNGER
...,...,...,...,...,...
3376,2021-04-14,5,52,8,UDSÆTNINGSGAST_D
3377,2021-04-14,5,55,9,PEJLEGAST_A
3378,2021-04-14,5,57,10,PEJLEGAST_B
3379,2021-04-14,5,41,11,DÆKSELEV_I_KABYS


## Load data

In [152]:
df = pd.read_csv('../data/synthetic.csv')

## Library

In [204]:
from enum import Enum
from itertools import product

import numpy as np
import pandas as pd
import pulp as P

class Task(Enum):
    VAGTHAVENDE_ELEV = 0
    ORDONNANS = 1
    UDKIG = 2
    BJÆRGEMÆRS = 3
    RORGÆNGER = 4
    UDSÆTNINGSGAST_A = 5
    UDSÆTNINGSGAST_B = 6
    UDSÆTNINGSGAST_C = 7
    UDSÆTNINGSGAST_D = 8
    PEJLEGAST_A = 9
    PEJLEGAST_B = 10
    DÆKSELEV_I_KABYS = 11

def team_minmax(team, size=20):
    return team*size, team*size+size

def get_task_counts(df_log, n_gasts=60):
    n_tasks = len(Task)
    hist = df[df.task_no >= 0].groupby(['gast_no', 'task_no']).size()
    stats = np.zeros((n_gasts, n_tasks)).astype(int)
    for gast_no, task_no in hist.index:
        stats[gast_no , task_no] = hist[gast_no, task_no]
    return stats

def get_instance(task_counts, vagthavende, pejlegast_b=None, sick=[], hu=[], assigned={}, n_shifts=6, pejl_shift=2, kabys_shifts=[0,1,2], team_order=[0,1,2]):
    """
    TODO: handle vagthavende samme morgen og aften
    TODO: håndter allerede udfyldte vagter
    """
    n_gasts, n_tasks = task_counts.shape
    n_teams = len(team_order)
    assert(len(vagthavende) == n_teams)
    
    # x_ijk, gast i, task j, shift k
    X = P.LpVariable.dicts('x', (range(n_gasts), range(n_tasks), range(n_shifts)), 0, 1, P.LpBinary)
    
    V = P.LpVariable.dicts('v', (range(n_gasts), range(n_teams)), 0, 1, P.LpBinary)

    prob = P.LpProblem('Schedule', P.LpMinimize)
    
    # objective function
    prob += P.lpSum([
        task_counts[i][j] * X[i][j][k] 
        for i in range(n_gasts) 
        for j in range(n_tasks) 
        for k in range(n_shifts)
    ])
    
    # same vagthavende gast morning and evening constraint
    # - Couldn't solve elegantly yet, so controlled via parameter for now
    # - probably not possible with a linear constraint on X; could introduce special variable
    j_vhe = Task.VAGTHAVENDE_ELEV.value
    for k1, team in enumerate(team_order):
        k2 = k1 + 3
        min_gast, max_gast = team_minmax(team)
        matches = list(filter(lambda i: min_gast <= i < max_gast, vagthavende))
        # check that exactly one gast from each team has vhe duty
        assert(len(matches) == 1)
        i = matches[0]
        prob += X[i][j_vhe][k1] == 1
        prob += X[i][j_vhe][k2] == 1        
            
    
    # pre-assigned duties constraints
    for k,value in assigned.items():
        for i,task in value.items():
            j = task.value
            prob += X[i][j][k] == 1
    
        
    # pejlegast constraints
    j_a = Task.PEJLEGAST_A.value
    j_b = Task.PEJLEGAST_B.value
    for k in range(n_shifts):
        if k == pejl_shift:
            if pejlegast_b is not None:
                # assert that pejlegast_b is from right tea
                active_team = team_order[k % 3]
                min_gast, max_gast = team_minmax(active_team)
                
                prob += X[pejlegast_b][j_b][k] == 1
            else:
                prob += P.lpSum([X[i][j_b][k] for i in range(n_gasts)]) == 1
            prob += P.lpSum([X[i][j_a][k] for i in range(n_gasts)]) == 1
        else:
            prob += P.lpSum([X[i][j_a][k] for i in range(n_gasts)]) == 0
            prob += P.lpSum([X[i][j_b][k] for i in range(n_gasts)]) == 0        
    
    # kabys constraint
    j = Task.DÆKSELEV_I_KABYS.value
    for k in range(n_shifts):
        if k in kabys_shifts:
            prob += P.lpSum([X[i][j][k] for i in range(n_gasts)]) == 1        
        else:
            prob += P.lpSum([X[i][j][k] for i in range(n_gasts)]) == 0
                
    # 0 or 1 task per gast, depending on shift constraint
    for k in range(n_shifts):
        active_team = team_order[k % 3]
        min_gast, max_gast = team_minmax(active_team)
        for i in range(n_gasts):
            i_works = int(min_gast <= i < max_gast)
            prob += P.lpSum([X[i][j][k] for j in range(n_tasks)]) <= i_works
    
    # exactly 1 gast per task per shift, but ignore vagthavende, pejlegast and kabys
    ignored = [
        Task.VAGTHAVENDE_ELEV.value,
        Task.PEJLEGAST_A.value, 
        Task.PEJLEGAST_B.value, 
        Task.DÆKSELEV_I_KABYS.value, 
    ]
    other_tasks = [j for j in range(n_tasks) if j not in ignored]
    for k in range(n_shifts):        
        for j in other_tasks:
            #print(f'[DEBUG] shift {k}, task {j}')
            prob += P.lpSum([
                X[i][j][k] 
                for i in range(n_gasts)
            ]) == 1
            
    # no tasks for left-out gasts (i.e. sick and hu)
    for i in sick + hu:
        prob += P.lpSum([
            X[i][j][k] 
            for j in range(n_tasks)
            for k in range(n_shifts)
        ]) == 0

    return prob, X
        

## Example

In [206]:
# simulate som sick gasts
TIDSPUNKTER = ['08 - 12', '12 - 16', '16 - 20', '20 - 24', '24 - 04', '04 - 08']

task_counts = get_task_counts(df)

n_gasts, n_tasks = task_counts.shape
n_shifts = 6

sick = [0,1,2]
hu = [4,5,8]
assigned = {0: {6: Task.ORDONNANS, 3: Task.BJÆRGEMÆRS}}

prob, X = get_instance(task_counts, [1,21,41], pejlegast_b=42, sick=sick, hu=hu, assigned=assigned)
#prob, X = get_instance(task_counts)

status = prob.solve()
print(f'Optimal? {"yes" if status == 1 else "no"}')
print(f'Infeasible? {"yes" if status == -1 else "no"}')
print()

for k in range(n_shifts):
    print(f'Vagt {TIDSPUNKTER[k]}')
    for j in range(len(Task)):
        for i in range(n_gasts):
            x = X[i][j][k]
            if x.varValue == 1:
                task_name = Task(j)
                count = task_counts[i][j]
                print(f'- Gast {i} er {task_name} ({count} før)')



Optimal? no
Infeasible? yes

Vagt 08 - 12
- Gast 1 er Task.VAGTHAVENDE_ELEV (1 før)
- Gast 6 er Task.ORDONNANS (3 før)
- Gast 10 er Task.UDKIG (3 før)
- Gast 3 er Task.BJÆRGEMÆRS (3 før)
- Gast 18 er Task.RORGÆNGER (2 før)
- Gast 7 er Task.UDSÆTNINGSGAST_A (2 før)
- Gast 13 er Task.UDSÆTNINGSGAST_B (1 før)
- Gast 15 er Task.UDSÆTNINGSGAST_C (1 før)
- Gast 9 er Task.UDSÆTNINGSGAST_D (3 før)
- Gast 11 er Task.DÆKSELEV_I_KABYS (1 før)
Vagt 12 - 16
- Gast 21 er Task.VAGTHAVENDE_ELEV (5 før)
- Gast 22 er Task.ORDONNANS (1 før)
- Gast 24 er Task.UDKIG (1 før)
- Gast 29 er Task.BJÆRGEMÆRS (1 før)
- Gast 35 er Task.RORGÆNGER (0 før)
- Gast 30 er Task.UDSÆTNINGSGAST_A (1 før)
- Gast 32 er Task.UDSÆTNINGSGAST_B (1 før)
- Gast 31 er Task.UDSÆTNINGSGAST_C (2 før)
- Gast 25 er Task.UDSÆTNINGSGAST_D (1 før)
- Gast 23 er Task.DÆKSELEV_I_KABYS (1 før)
Vagt 16 - 20
- Gast 41 er Task.VAGTHAVENDE_ELEV (12 før)
- Gast 48 er Task.ORDONNANS (1 før)
- Gast 49 er Task.UDKIG (2 før)
- Gast 44 er Task.BJÆRGEMÆR

## Brugbare snippets

In [157]:
import re
for w in re.findall(r"[\w']+", '1,2,3'):
    print(w)


1
2
3


In [186]:
list(filter(lambda x: x % 2 == 0, [1,2,3]))

[2]